#DeepForest Training Walkthrough(CPU/GPU)
  - For GPU implementation.
      1. Select **Runtime** > Change **runtime type** and Select GPU as Hardware accelerator.

In [7]:
#installing deepforest (Different from original as this gives ability to vary thickness of predictions)
! pip install git+https://github.com/weecology/deepforest.git@refs/pull/240/head


# you can simply use pip install deepforest i used beacause you can find explaination on this link = https://github.com/weecology/DeepForest/issues/239


!pip install albumentations==0.4.6
import albumentations 


from albumentations.pytorch import ToTensorV2

  Cloning https://github.com/weecology/deepforest.git (to revision refs/pull/240/head) to /tmp/pip-req-build-rvu5jlyt
  Running command git clone -q https://github.com/weecology/deepforest.git /tmp/pip-req-build-rvu5jlyt
  Running command git fetch -q https://github.com/weecology/deepforest.git refs/pull/240/head
  Running command git checkout -q a66f0fe05edcba4732c2be005e4b4478fbbac9a6


# DeepForest Import 

In [5]:
#load the modules
import os
import time
import numpy as np
from deepforest import main 
from deepforest import get_data
from deepforest import utilities
from deepforest import preprocess

from deepforest import main
from deepforest import get_data
import os
import matplotlib.pyplot as plt
model = main.deepforest()
model.use_release()


Reading config file: /usr/local/lib/python3.7/dist-packages/deepforest/data/deepforest_config.yml


Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_coco-eeacb38b.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_coco-eeacb38b.pth


  0%|          | 0.00/130M [00:00<?, ?B/s]

NEON.pt: 129MB [00:02, 48.4MB/s]                           


Model was downloaded and saved to /usr/local/lib/python3.7/dist-packages/deepforest/data/NEON.pt
Loading pre-built model: https://github.com/weecology/DeepForest/releases/tag/1.0.0


# Loading Custom trained model (Model which trained on boston specific dataset)

In [ ]:
#load the save model, trained on boston dataset
import torch
model = main.deepforest()
model_path='/content/drive/MyDrive/GSoC 2021 Final/70epchs.pth'  #path to pretrained model 
model.model.load_state_dict(torch.load(model_path))

If you have large tile lets assume 22k * 22k then you cannot predict on entire large file hence we made patches of that large file and then run our DeepForest model over that patches. 

Next cell contain code for making patches and saving those patches over drive

In [ ]:
from osgeo import gdal
from osgeo.gdalconst import *
import numpy as np

import osgeo.gdalnumeric as gdn
import matplotlib.pyplot as plt

In [ ]:
import os, sys
from osgeo import gdal
#inpath is where the bigger TIFF file is stored
#outpath where we will be saving clipped smaller tiles
inpath="/content/drive/MyDrive/data/Data/Copy of Boston_MA_Nearmap_Summer20_154.tif"
outpath="/content/drive/MyDrive/data/tiles154/"
dset = gdal.Open(inpath)

width = dset.RasterXSize
height = dset.RasterYSize

tilesize = 3000 #this tile size is user defined you need to perform multiple iteration to understand which patch size will be better for your particular dataset. patch size will depend on various factors such as Ground truth Resoultion .

print(dset)

In [ ]:
#To split a larger tile, into smaller tiles of size specified above
for i in range(0, width, tilesize):
    for j in range(0, height, tilesize):
        w = min(i+tilesize, width) - i
        h = min(j+tilesize, height) - j
        gdaltranString = "gdal_translate -of GTIFF -srcwin "+str(i)+", "+str(j)+", "+str(w)+", " \
            +str(h)+" " + str(inpath) + " " + str(outpath) + 'tile_'+ "_"+str(i)+"_"+str(j)+".tif"
        os.system(gdaltranString)

# Predictions

This is code to predict canopies over 3000 * 3000 patches. This is iterative code as our original dataset consist of 22000 * 22000 patch and we made its patches of 3000 * 3000 . And we did predictions on all patches hence we write iterative code to get predictions over all patches. You can use this code for single patch also.


In [ ]:
#Read the clipped images(samller tiles) from the directory
directory=outpath[:-1]
fic=0
i=0
from osgeo import gdal
from osgeo import ogr
from osgeo import osr
import pandas as pd
df = pd.DataFrame(columns = ['treecount',    'filename'  , 'location', 'lat', 'long' ])

for filename in os.listdir(directory):
            if filename.endswith(".tif") : #or filename.endswith(".png")
                
                filep = str(os.path.join(directory, filename))
                


                
                inpath=filep
                dset = gdal.Open(inpath)
                if (dset.RasterXSize >=3000 and dset.RasterYSize >=3000):
                  print("Corresponding tile/filename:",filename)
                  img = model.predict_image(path=filep,return_plot=True, color=(0, 0, 200), thickness=5)
                  box= model.predict_image(path=filep,return_plot=False)

                  fic = fic + 1
                  plt.figure(figsize=(8,8))
                  plt.imshow(img[:,:,::-1])
                  plt.show()

                  treecounttm=box.shape[0]

                  
                  #dataframe creation
                  src = gdal.Open(filep)
                  ulx, xres, xskew, uly, yskew, yres  = src.GetGeoTransform()
                  lrx = ulx + (src.RasterXSize * xres)
                  lry = uly + (src.RasterYSize * yres)
                  # print(lrx,lry,ulx,uly)
                  mpx=0.5*(lrx+ulx)
                  mpy=0.5*(lry+uly)


                  # Setup the source projection - you can also import from epsg, proj4...
                  source = osr.SpatialReference()
                  source.ImportFromWkt(src.GetProjection())

                  # The target projection
                  target = osr.SpatialReference()
                  target.ImportFromEPSG(4326)

                  # Create the transform - this can be used repeatedly
                  transform = osr.CoordinateTransformation(source, target)

                  # Transform the point. You can also create an ogr geometry and use the more generic `point.Transform()`
                  latlong=transform.TransformPoint(mpx, mpy)
                  # print(latlong)
                  print(f"Canopies detected according to trained model {treecounttm} and correspond to tile with latitude and longitude {latlong[1]} , {latlong[0]}")



                  location=[latlong[1], latlong[0]]
                  df.loc[i] = [treecounttm, filename, location, latlong[1],latlong[0]]             



                  i=i+1
                  print(" ")


print("END")



# Visualization over

This visuzation only possible when you are having Geotiff files as input beacause we are using here location information from Geotiff file for doing visulaization over Google maps.


In [ ]:
#export predictions to CSV for a tile
df.to_csv('/content/'+'predictionsfortile.csv')

In [ ]:
!pip install rasterio
import rasterio

from osgeo import gdal

# Import necessary packages
import os 
import folium
from folium import plugins

location=df['location'][int(len(df)*0.5)]
print(location)

In [ ]:
randomtile=location

In [ ]:
map_osm = folium.Map(
    location = randomtile,
    zoom_start = 12
)



In [ ]:
#Mark the points in Map
for indice, row in df.iterrows():
  # print(row["lat"], row["long"])
  folium.Marker(
      location=[row["lat"], row["long"]],
      popup=folium.Popup(str(row['filename']+str('=')+str(row["treecount"])),show=True)
  ).add_to(map_osm)

# print(row["lat"], row["long"])
map_osm